In [39]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import scipy.integrate as integrate
import yaml
import pandas as pd
from scipy.optimize import curve_fit # for fitting the density with a function

## Constants

In [40]:
MEarth = 5.972e24 #kg
R_Earth = 6371 #km
G = 6.67384e-11 #m3/kg/s2
year = 365.25*3600*24 #s

## Rocky planet class

Defines physical parameters and evolution of a given planet

In [46]:
class Rocky_Planet():

    def __init__(self):
        self.parameters()

    def parameters(self):
        pass # parameters are defined in class

    def A_rho(self):
        return (5. * self.Kprime_0 - 13.) / 10.
    
    def evolution(self):
        evolution = Evolution(self)
        evolution.energies()
        evolution.profiles()

    def read_parameters(self, file):
        with open(file, 'r') as stream:
            try:
                dict_param = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
        for k, v in dict_param.items():
            setattr(self, k, float(v))
            
    def open_data_profiles(filename, core=False):
        names = ["g(m/s^2)", "p(GPa)", "rho(kg/m^3)","r(m)", "T(K)", "oups", "Cp(J/kgK)", "alpha(10^-5 1/s)", "Gruneisen(1)", \
                 "KT(GPa)", "KS(GPa)", "G(GPa)", "ElCond (Siemens)", "Material-Parameter" ]
        data = pd.read_csv(filename, skipinitialspace=True, sep=" ",
                           names=names, index_col=False)
        if core == True:
            data = data[data["Material-Parameter"]==8.]
        return data

    def density_Labrosse2015(r, *args):
        """ Equation (5) in Labrosse 2015 """
        rho_0, L_rho, A_rho = args
        return rho_0*(1-r**2/L_rho**2-A_rho*r**4/L_rho**4)

    def find_Lrho_Arho(profile):
        """ least square to fit the function density to the data.count

        curve_fit use non-linear least square,
        the default method is Levenberg-Marquardt algorithm as implemented in MINPACK.
        (doesn't handle bounds and sparse jacobians. Should be enough here)
         """
        rho = profile["rho(kg/m^3)"]
        radius = profile["r(m)"]
        initial_guess = 12500, 8000e3, 0.484 # initial values from Labrosse 2015 (for the Earth)
        popt, pcov = curve_fit(density_Labrosse2015, radius, rho, initial_guess)
        rho_0, L_rho, A_rho = popt
        return rho_0.tolist(), L_rho.tolist(), A_rho.tolist()

    def name_file(XFe, Mp, FeM):
        return "data_prof_M_ {:.1f}_Fe_{:.0f}.0000_FeM_{:2.0f}.0000.res".format(Mp, XFe, FeM)

    def Earth():
        """ Default values for Earth (see Labrosse 2015) """
        param = {
                        "r_OC" : 3480.e3,          # Core radius (m)
                        "r_IC" : 1221.e3,          # Present inner core radius (m)
                        "r_IC_0": 0., # Initial inner core radius (m)
                        #
                        "alpha_c" : 1.3e-5,           # Thermal expansion coefficient at center (K-1)
                        "CP" : 750.,             # Specific heat (J.kg-1.K-1)
                        "gamma" : 1.5,              # Grueneisen parameter
                        # Parameters obtained from the fitting of the density
                        "rho_0" :  12502.,           # Density at center (kgm-3)
                        "L_rho" : 8039e3,             # (m)
                        "A_rho" : 0.484,              # (no unit)
                        # Parameter obtained from the qs_**.res  
                        "Q_CMB" : 7.4e12,           # Present CMB flux (W)
                        # Parameters not expected to be modified
                        "Deltarho_ICB" : 500.,             # Density jump at ICB (kgm-3)
                        "DeltaS" : 127.,             # Entropy of crystallization (Jkg-1K-1)
                        "H" : 0.,               # Radioactivity
                        "k_c" : 163.,             # Thermal conductivity at center (Wm-1K-1)
                        "beta" : 0.83,           # coefficient of compositional expansion
                        "chi0" : 0.056,            # Difference in mass fraction of light elements across ICB
                        # Phase diag (maybe not modified?)
                        "dTL_dchi" : -21.e3,           # Compositional dependence of liquidus temperature (K)
                        "dTL_dP" : 9.e-9,            # Pressure dependence of liquidus temperature (Pa-1)
                        "TL0" : 5700.,            # Melting temperature at center (K)
            }
        return param

    def average_volume(profile, name_variable):
        """ Average of the parameter called name_variable over the full volume """
        profile["dV"] = 4*np.pi*profile["r(m)"]
        volume = profile["dV"].sum()
        quantity = profile["dV"]*profile[name_variable]
        quantity_total = quantity.sum()
        return quantity_total/volume

    def write_parameter_file(XFe, Mp, FeM):
        """ write the yaml parameter file  """
        filename = name_file(XFe, Mp, FeM)
        core = open_data_profiles(filename, core=True)

        # initialisation with Earth parameters
        param = Earth()

        # modification of the parameters from the profiles
        param["rho_0"], param["L_rho"], param["A_rho"] = find_Lrho_Arho(core)
        param["r_OC"] = core["r(m)"].iloc[-1].tolist()
        param["CP"] = average_volume(core, "Cp(J/kgK)").tolist()
        param["alpha_c"] = average_volume(core, "alpha(10^-5 1/s)").tolist() * 1e-5
        param["gamma"] = average_volume(core, "Gruneisen(1)").tolist()
        output_filename = "M_ {:.1f}_Fe_{:.0f}.0000_FeM_{:2.0f}.0000.yaml".format(Mp, XFe, FeM)

        # write yaml file
        with open(output_filename, 'w') as outfile:
            yaml.dump(param, outfile, default_flow_style=False)


#write_parameter_file(30, 1.2, 0)
#write_parameter_file(50, 1.2, 0)

## Evolution class

Calculates the thermal evolution of a given planet

In [44]:
class Evolution():

    def __init__(self, planet):
        self.planet = planet
        
    def calc_L_T(self):
        """ Temperature change length scale """
        return np.sqrt(3.*self.planet.CP/(2.*np.pi*self.planet.alpha_c*self.planet.rho_c*self.planets.GC))
        
    def calc_L_rho(self):
        """ Density change length scale """
        return np.sqrt(3.*self.planet.K_c/(2.*np.pi*G*self.planet.rho_0*self.planet.rho_c)*(np.log(self.planet.rho_c/self.planet.rho_0)+1.))

    def dTL_dr_IC(self, r):
        ''' Melting temperature jump at ICB '''
        return -self.planet.K_c * 2.*self.planet.dTL_dP * r / self.planet.L_rho**2. \
            + 3. * self.planet.dTL_dchi * self.planet.chi0 * r**2. / (self.planet.L_rho**3. * self.fC(self.planet.r_OC / self.planet.L_rho, 0.))

    def fC(self, r, delta): 
        '''fC (Eq. A1 Labrosse 2015)'''
        return r**3. * (1 - 3. / 5. * (delta + 1) * r**2.- 3. / 14. * (delta + 1) \
            * (2 * self.planet.A_rho - delta) * r**4.)

    def fX(self, r, r_IC):
        '''fX (Eq. A15 Labrosse 2015)'''
        return (r)**3. * (-r_IC**2. / 3. / self.planet.L_rho**2. + 1./5. * (1.+r_IC**2./self.planet.L_rho**2.) \
                *(r)**2.-13./70. * (r)**4.) 

    def rho(self, r):
        ''' Density (Eq. 5 Labrosse 2015)'''
        return self.planet.rho_c * (1. - r**2. / self.planet.L_rho**2. - self.planet.A_rho * r**4. / self.planet.L_rho**4.)

    def T_melt(self, r):
        ''' Melting temperature at ICB (Eq. 14 Labrosse 2015)'''
        return self.planet.TL0 - self.planet.K_c * self.planet.dTL_dP * r**2. / self.planet.L_rho**2. + self.planet.dTL_dchi * self.planet.chi0 * r**3. \
                / (self.planet.L_rho**3. * self.fC(self.planet.r_OC / self.planet.L_rho, 0.))

    def PL(self, r):
        '''Latent heat power'''
        return 4. * np.pi * r**2. * self.T_melt(r) * self.rho(r) * self.planet.DeltaS

    def LH(self, r):
        LH, i = integrate.quad(self.PL, 0, r)
        return LH

    def Pc(self, r):
        '''Secular cooling power (Eq. A8 Labrosse 2015)'''
        return -4. * np.pi / 3. * self.planet.rho_c * self.planet.CP * self.planet.L_rho**3. *\
                (1 - r**2. / self.planet.L_rho**2 - self.planet.A_rho* r**4. / self.planet.L_rho**4.)**(-self.planet.gamma) \
                * (self.dTL_dr_IC(r) + 2. * self.planet.gamma \
                * self.T_melt(r) * r / self.planet.L_rho**2. *(1 + 2. * self.planet.A_rho * r**2. / self.planet.L_rho**2.) \
                /(1 - r**2. / self.planet.L_rho**2. - self.planet.A_rho * r**4. / self.planet.L_rho**4.)) \
                * (self.fC(self.planet.r_OC / self.planet.L_rho, self.planet.gamma))

    def SC(self, r):
        SC, i = integrate.quad(self.Pc, 0, r)  
        return SC

    def Px(self, r):
        ''' Gravitational heat power (Eq. A14 Labrosse 2015)'''
        return 8 * np.pi**2 * self.planet.chi0 * self.planet.GC * self.planet.rho_c**2 * self.planet.beta * r**2. \
        * self.planet.L_rho**2. / self.fC(self.planet.r_OC / self.planet.L_rho, 0) \
        * (self.fX(self.planet.r_OC / self.planet.L_rho, r) - self.fX(r / self.planet.L_rho, r))

    def GH(self, r):
        GH, i = integrate.quad(self.Px, 0, r)
        return GH


    def energies(self):
        ''' Latent heat '''
        self.L = 4. * np.pi / 3. * self.planet.rho_c * self.planet.TL0 * self.planet.DeltaS * self.planet.r_IC**3. * (1 - 3. / 5. \
            * (1 + self.planet.K_c / self.planet.TL0 * self.planet.dTL_dP) * self.planet.r_IC**2. / self.planet.L_rho**2. \
            + self.planet.chi0 / (2 * self.fC(self.planet.r_OC / self.planet.L_rho, 0.) * self.planet.TL0) * self.planet.dTL_dchi * self.planet.r_IC**3. / self.planet.L_rho**3.)
        print("Latent heat", self.L,"J")
        
        ''' Secular cooling '''
        self.C = 4. * np.pi / 3. * self.planet.rho_c * self.planet.CP * self.planet.L_rho * self.planet.r_IC**2 * self.fC(self.planet.r_OC / self.planet.L_rho, self.planet.gamma)\
                * (self.planet.dTL_dP * self.planet.K_c - self.planet.gamma * self.planet.TL0 - self.planet.dTL_dchi * self.planet.chi0 / self.fC(self.planet.r_OC / self.planet.L_rho, 0.) * self.planet.r_IC / self.planet.L_rho)    
        print("Secular cooling", self.C,"J")
        self.C, i = integrate.quad(self.Pc, 0, self.planet.r_IC)
        # Why are you calculating C twice??
        print("Secular cooling", self.C,"J")

        ''' Gravitational energy '''
        self.G = 8 * np.pi**2. / 15. * self.planet.chi0 * self.planet.GC * self.planet.rho_c**2. * self.planet.beta * self.planet.r_IC**3. * self.planet.r_OC**5. / self.planet.L_rho**3. \
            / self.fC(self.planet.r_OC/self.planet.L_rho,0)*(1. - self.planet.r_IC**2 / self.planet.r_OC**2 + 3. * self.planet.r_IC**2. / 5. / self.planet.L_rho**2. \
                - 13. * self.planet.r_OC**2. / 14. / self.planet.L_rho**2. + 5./18. * self.planet.r_IC**3. * self.planet.L_rho**2. /self.planet.r_OC**5.)
        print("Gravitational energy", self.G,"J")

        ''' Total energy '''
        self.E_tot = self.L + self.C + self.G
        print("Total energy", self.E_tot,"J")


    def profiles(self, N=50):

        r_IC_vec = np.linspace(0, self.planet.r_IC, N)
        Q_cmb = np.linspace(3, 10, 4)*1e12   # From Labrosse+2001

        L_H = np.zeros(N)
        S_C = np.zeros(N)
        G_H = np.zeros(N)
        time = np.zeros(N)

        for i, radius_ic in enumerate(r_IC_vec):
            L_H[i] = self.LH(radius_ic)
            S_C[i] = self.SC(radius_ic)
            G_H[i] = self.GH(radius_ic)

        E_tot = L_H + S_C + G_H

        time = E_tot / self.planet.Q_CMB/(np.pi*1e7*1e9) # Q_CMB is a specific value!
        
        
        plt.figure(1)
        label = ['Latent heating','Secular cooling','Gravitational heating','Total energy']
        plt.plot(time, L_H,label=label[0])
        plt.plot(time, S_C,label=label[1]) 
        plt.plot(time, G_H, label=label[2])
        plt.plot(time, E_tot,label=label[3])
        plt.legend()
        plt.xlabel('Time (Gyr)')
        plt.ylabel('Energy (J)')

        plt.figure(2)
        plt.plot(time, r_IC_vec*1e-3,label='$Q_{\mathrm{CMB}}$=7.4e12 W')
        plt.xlabel('Time (Gyr)')
        plt.ylabel('Inner core size (km)')
        plt.legend()

        L_H= np.zeros((N,4),dtype=np.float32)
        S_C = np.zeros((N,4),dtype=np.float32)
        G_H = np.zeros((N,4),dtype=np.float32)
        time = np.zeros((N,4),dtype=np.float32)

        for i in range(len(r_IC_vec)):
            L_H[i] = self.LH(r_IC_vec[i])
            S_C[i] = self.SC(r_IC_vec[i])
            G_H[i] = self.GH(r_IC_vec[i])

        E_tot = L_H + S_C + G_H

        time = (L_H + S_C + G_H) / Q_cmb/(np.pi*1e7*1e9)

        plt.figure(3)
        plt.plot(time, r_IC_vec*1e-3)
        plt.xlabel('Time (Gyr)')
        plt.ylabel('Inner core size (km)')
        plt.show()